In [2]:
from langchain.llms import GooglePalm
from hello import api_key
import os
os.environ['GOOGLE_AI_KEY']=api_key
llm=GooglePalm(google_api_key=api_key,temperature=0.6)

c:\Kriti\college\codefile\inhouse\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path="project.csv", source_column="States + UTs")

data = loader.load()





In [15]:
import streamlit as st
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
e = embeddings.embed_query("states")
len(e)

instructor_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_query"
)
vectordb = FAISS.from_documents(documents=data,embedding=instructor_embeddings)


retriever = vectordb.as_retriever(score_threshold = 0.7)
rdocs = retriever.get_relevant_documents("Goa")
rdocs


from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
chain = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriever, input_key="query",return_source_documents=True, chain_type_kwargs=chain_type_kwargs)
chain("spirtuality of Goa and its origin")




{'query': 'spirtuality of Goa and its origin',
 'result': 'The spirituality of Goa is Christian Contemplative Prayer, which originated from Christianity.',
 'source_documents': [Document(page_content='S No.: 6\nStates + UTs: Goa\nCapitals: Panaji\nRegion: West\nPractice Name: Christian Contemplative Prayer\nOrigin: Christianity\nFocus: Spiritual intimacy with God\nMethod: Silent prayer and reflection focusing on divine presence\nFood: Vindaloo\nIngredients: Chicken, coconut oil, wine vinegar, ginger, green, cinnamon\nDiet: non vegetarian\nPrep_time: 10\nCook_time: 40\nFlavor_profile: spicy\nCourse: main course', metadata={'source': 'Goa', 'row': 34}),
  Document(page_content='S No.: 3\nStates + UTs: Dadra and Nagar Haveli and Daman and Diu\nCapitals: Daman\nRegion: West\nPractice Name: Tribal Rituals\nOrigin: Indigenous Beliefs\nFocus: Cultural spirituality\nMethod: Traditional dances, songs, and rituals\nFood: Ghooghra\nIngredients: Semolina, coconut, cardamom, sugar\nDiet: vegetarian